In [ ]:
%pip install cryptography

### 1. Hàm băm dữ liệu và hàm tính giá trị h0


In [14]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes

def hash_data(data):  
    hasher = hashes.Hash(hashes.SHA256(), backend=default_backend())
    hasher.update(data)
    return hasher.finalize()

def calculate_h0(file_path):
    with open(file_path, "rb") as f:
        blocks = []
        
        block = f.read(1024)

        while block:
            blocks.append(block)
            block = f.read(1024)
        h = [None] * len(blocks)

        h[-1] = hash_data(blocks[-1])
        
        for i in range(len(blocks)-2,-1,-1):
            h[i] = hash_data(blocks[i] + h[i+1])
        return h[0]

file_path = "C:\\Users\\ADMIN\\Downloads\\birthday.mp4"

hash_value =  calculate_h0(file_path)

print("Giá trị H0 sau SHA256:", hash_value.hex())


SHA256 H0 Value: 03c08f4ee0b576fe319338139c045c89c3e8e9409633bea29442e21425006ea8


### 2. Hàm băm file

In [39]:
def block_hash_video(input_video_path, output_video_path):
    block_size = 1024 

    with open(input_video_path, "rb") as file_in:
        blocks = []
        block = file_in.read(block_size)  
        
        while block:
            blocks.append(block)
            block = file_in.read(1024)

        h = [None] * len(blocks)
        h[-1] = hash_data(blocks[-1])

        for i in range(len(blocks)-2,-1,-1):
            h[i] = hash_data(blocks[i] + h[i+1])

        for i in range(0,len(blocks)-1,1):
            with open(f"{output_video_path}\\block_{i}.mp4", "wb") as file_out:
                
                print(f"{output_video_path}\\block_{i}.mp4")
                file_out.write(blocks[i])
                file_out.write(h[i+1])
                
        with open(f"{output_video_path}\\block_{len(blocks)-1}.mp4", "wb") as file_out:
                file_out.write(blocks[-1])

        print(f"Hash file: {output_video_path}")
        return h[0].hex()

### 3. Tiến hành thử băm video với mỗi block 1kb, thu được 16530 file block trong thư mục birthday


In [74]:
input_video_path = "C:\\Users\\ADMIN\\Downloads\\birthday.mp4"
output_video_path = "C:\\Users\\ADMIN\\Downloads\\birthday"
block_hash_video(input_video_path,output_video_path)

C:\Users\ADMIN\Downloads\birthday\block_0.mp4
C:\Users\ADMIN\Downloads\birthday\block_1.mp4
C:\Users\ADMIN\Downloads\birthday\block_2.mp4
C:\Users\ADMIN\Downloads\birthday\block_3.mp4
C:\Users\ADMIN\Downloads\birthday\block_4.mp4
C:\Users\ADMIN\Downloads\birthday\block_5.mp4
C:\Users\ADMIN\Downloads\birthday\block_6.mp4
C:\Users\ADMIN\Downloads\birthday\block_7.mp4
C:\Users\ADMIN\Downloads\birthday\block_8.mp4
C:\Users\ADMIN\Downloads\birthday\block_9.mp4
C:\Users\ADMIN\Downloads\birthday\block_10.mp4
C:\Users\ADMIN\Downloads\birthday\block_11.mp4
C:\Users\ADMIN\Downloads\birthday\block_12.mp4
C:\Users\ADMIN\Downloads\birthday\block_13.mp4
C:\Users\ADMIN\Downloads\birthday\block_14.mp4
C:\Users\ADMIN\Downloads\birthday\block_15.mp4
C:\Users\ADMIN\Downloads\birthday\block_16.mp4
C:\Users\ADMIN\Downloads\birthday\block_17.mp4
C:\Users\ADMIN\Downloads\birthday\block_18.mp4
C:\Users\ADMIN\Downloads\birthday\block_19.mp4
C:\Users\ADMIN\Downloads\birthday\block_20.mp4
C:\Users\ADMIN\Download

'03c08f4ee0b576fe319338139c045c89c3e8e9409633bea29442e21425006ea8'

### 4. Hàm kiểm tra video là hợp lệ với các giá trị h0,h1,h2,..

In [72]:
import os
def check_data(block, h_block, h_next_block):  
    hasher = hashes.Hash(hashes.SHA256(), backend=default_backend())
    hasher.update(block + h_next_block)
    return hasher.finalize() == h_block

def check_video_part(file_video_block_path, h_block):
    with open(file_video_block_path, "rb") as f:
        block = f.read(1024)
        h_next_block = f.read(32)
        return [check_data(block,h_block,h_next_block),h_next_block]
    
def check_all_video(file_video_folder_path,h0):
    video_files = []
    total_video = len(os.listdir(file_video_folder_path))
    video_valid = 1
    for i in range(0,total_video-1):
        video_files.append(file_video_folder_path +"\\block_{}.mp4".format(i))

    h_block = h0
    for video_path in video_files:
        is_valid, h_block = check_video_part(video_path,h_block)
        # Kiểm tra block hợp lệ và tính toán h_block cho block tiếp theo
        if(is_valid):
            print(video_path + ' - ' + "is valid")
        else :
            print("Warning! " + video_path + ' - ' + "IS  INVALID")
            print("Stop checking video!")
            video_valid = 0
            break
    if (video_valid==1):
        print("DONE! ALL VIDEO IS VALID")



### 5. Chạy thử quá trình kiểm tra với file hoàn chỉnh 

In [75]:
check_all_video(output_video_path,hash_value)

C:\Users\ADMIN\Downloads\birthday\block_0.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_1.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_2.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_3.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_4.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_5.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_6.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_7.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_8.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_9.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_10.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_11.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_12.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_13.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_14.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_15.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_16.mp4 - is valid
C:\Users\ADMIN\Downloads

### 6. Chạy thử hàm kiểm tra với file bị sửa đổi tại block 100

In [76]:
import random

num_bytes_to_modify = 10
with open("C:\\Users\\ADMIN\\Downloads\\birthday\\block_100.mp4",'r+b') as file:
    data = file.read()
    positions = random.sample(range(len(data)), num_bytes_to_modify)
    for pos in positions:
        file.seek(pos)
        random_byte = bytes([random.randint(0, 255)])
        file.write(random_byte)

check_all_video(output_video_path,hash_value)

C:\Users\ADMIN\Downloads\birthday\block_0.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_1.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_2.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_3.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_4.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_5.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_6.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_7.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_8.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_9.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_10.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_11.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_12.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_13.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_14.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_15.mp4 - is valid
C:\Users\ADMIN\Downloads\birthday\block_16.mp4 - is valid
C:\Users\ADMIN\Downloads